<a href="https://colab.research.google.com/github/dimarsoft/ColabYOLO/blob/main/dimarsoft_yolov8_tracking_Track_YOLOv8x_detect_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA%D0%B0%D1%81%D0%BE%D0%BA_%D0%B8_%D0%B6%D0%B8%D0%BB%D0%B5%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Подключаем диск: на диске видео и потом туда сохраняем labels и видео с результатами.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Установка yolo8

In [ ]:
!git clone --recurse-submodules https://github.com/dimarsoft/yolov8_tracking.git  # clone recursively
%cd yolov8_tracking
!pip install -r requirements.txt  # install dependencies

Далее важно запустить mytrack.py, внем есть исправления для сохранения класса объекта

In [ ]:
!python mytrack.py --save-trajectories --save-txt --save-vid --source '/content/drive/MyDrive/AI_2023/dataset-v1.1/test/11.mp4' --yolo-weights '/content/drive/MyDrive/AI_2023/TrainedModels/Yolov8/640img_8x_best_b16_e10.pt'

mytrack: yolo_weights=[PosixPath('/content/drive/MyDrive/AI_2023/TrainedModels/Yolov8/640img_8x_best_b16_e10.pt')], reid_weights=/content/yolov8_tracking/weights/osnet_x0_25_msmt17.pt, tracking_method=bytetrack, tracking_config=trackers/bytetrack/configs/bytetrack.yaml, source=/content/drive/MyDrive/AI_2023/dataset-v1.1/test/11.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=False, save_txt=True, save_conf=False, save_crop=False, save_trajectories=True, save_vid=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, vid_stride=1, retina_masks=False
Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients, 257.4 GFLOPs
video 1/1 (1/584) /content/drive/MyDrive/AI_2023/dat

На выходе имеем runs/track/exp
11.mp4 видео с рамками, но с учетом обработки траектории (по моему)
11.txt траектория по входному видео.
формат файла - MOT format, что это пока не знаю

Возможно:
https://motchallenge.net/instructions/

`<frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <x>, <y>, <z>`

https://drive.google.com/drive/folders/1C9EnkkTnxzVMiTG1Gsu4bODUWgcEByOW?usp=sharing



In [ ]:
from pathlib import Path
import cv2
from enum import Enum
import argparse

import numpy as np


class Labels(Enum):
    human = 0
    helmet = 1
    uniform = 2


# турникет, потом вынести в настройку
line_a = -0.2
line_b = 0.68

# цвета обектов

label_colors = {
    Labels.human: (255, 255, 0),
    Labels.helmet: (255, 0, 255),
    Labels.uniform: (255, 255, 255)
}


class DetectedLabel:
    def __init__(self, label, x, y, width, height):
        self.label = label
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.above = None

    def label_str(self):
        if self.label is Labels.human:
            return "human"
        if self.label is Labels.uniform:
            return "uniform"
        if self.label is Labels.helmet:
            return "helmet"
        return ""


class DetectedTrackLabel(DetectedLabel):
    def __init__(self, label, x, y, width, height, track_id, frame):
        super(DetectedTrackLabel, self).__init__(label, x, y, width, height)

        self.track_id = track_id
        self.frame = frame - 1

    def get_caption(self):
        return f"{self.track_id}: {self.label_str()}"



# получить координату y для х
def get_y(x):
    return line_a * x + line_b


# проверка человека: выше/ниже турникета
def test_human(label):
    y_turniket = get_y(label.x)

    if y_turniket > label.y:
        label.above = True
    else:
        label.above = False



def draw_track_on_frame(frame, draw_rect, frame_w, frame_h, frame_info: DetectedTrackLabel):
    # турникет рисуем один раз

    y1 = int(get_y(0) * frame_h)
    y2 = int(get_y(1) * frame_h)
    cv2.line(frame, (0, y1), (frame_w, y2), (0, 0, 255), 1)

    # if frame_info.labels is not None:
    lab = frame_info

    humans = 0

    hh = int(lab.height * frame_h)
    ww = int(lab.width * frame_w)

    x = int(lab.x * frame_w - ww / 2)
    y = int(lab.y * frame_h - hh / 2)

    cv2.putText(frame, frame_info.get_caption(), (x, y), 0, 1, (255, 255, 255), 2, cv2.LINE_AA)


    # рамка объекта

    if draw_rect:
        cv2.rectangle(frame, (x, y), (x + ww, y + hh), label_colors[lab.label], 1)

    # если человек, то рисуем центр масс
    if lab.label is Labels.human:
        x = int(x + ww / 2)
        y = int(y + hh / 2)

        if lab.above is True:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)

        cv2.circle(frame, (x, y), 10, color, -1)



# разбор строки: получаем тип и координаты bb
def parse_track_row(info, w, h):
    box = info.split(" ")
    if np.char.isnumeric(box[0].replace('\n', '')):
        width = float(box[4]) / w
        height = float(box[5]) / h

        x_center = float(box[2]) / w + width / 2
        y_center = float(box[3]) / h + height / 2

        return DetectedTrackLabel(Labels(int(box[10])),
                                  x_center, y_center, width, height,
                                  int(box[1]), int(box[0]))

    return None


def parse_track_txt(labels_path, suffix, w, h):
    labels = []
    path_to_track_txt = Path(labels_path) / f"{suffix}.txt"
    try:
        with open(path_to_track_txt, 'r') as handle:
            box_info = handle.readlines()
            for txt in box_info:
                lab = parse_track_row(txt, w, h)
                if lab is not None:
                    if lab.label is Labels.human:
                        test_human(lab)
                    labels.append(lab)
            return labels
    except FileNotFoundError as e:
        print(path_to_track_txt, e)
    return labels


def draw_track_on_video(src_video_path, output_video_path, labels_path):
    path_v = Path(src_video_path)
    suffix = path_v.stem

    # reading the input
    input_video = cv2.VideoCapture(src_video_path)

    fps = int(input_video.get(cv2.CAP_PROP_FPS))
    # ширина
    w = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    # высота
    h = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # количесто кадров в видео
    frames_in_video = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"input = {src_video_path}, w = {w}, h = {h}, fps = {fps}, frames_in_video = {frames_in_video}")

    tracks_info = parse_track_txt(labels_path, suffix, w, h)

    print(f"lines in track file = {len(tracks_info)}")

    output_video = cv2.VideoWriter(
        output_video_path, cv2.VideoWriter_fourcc(*'mp4v'),
        fps, (w, h))

    image_frames = []

    # считываем все фреймы из видео
    for i in range(frames_in_video):
        ret, frame = input_video.read()
        image_frames.append(frame)

    # наложение трека

    for track in tracks_info:
        draw_track_on_frame(image_frames[track.frame], True, w, h, track)

    # запись в выходной файл
    for i in range(frames_in_video):
        output_video.write(image_frames[i])

    output_video.release()
    input_video.release()


# пример запуска в питоне
def run_example():
    src_video_path = "/content/drive/MyDrive/AI_2023/dataset-v1.1/test/11.mp4"
    output_video_path = "/content/yolov8_tracking/runs/track/exp/11_post.mp4"
    labels_path = "/content/yolov8_tracking/runs/track/exp/tracks"

    draw_track_on_video(src_video_path, output_video_path, labels_path)


run_example()    


input = /content/drive/MyDrive/AI_2023/dataset-v1.1/test/11.mp4, w = 640, h = 640, fps = 12, frames_in_video = 584
